In [53]:
import csv
import pandas as pd
import pickle
import re
from sklearn.model_selection import train_test_split
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
import numpy as np
import os.path
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import cross_validate
from scipy.sparse import hstack

In [2]:
data = pd.read_excel("Rest_Mex_2022.xlsx")
data.head(10)

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen dos veces antes de ir a este hotel, te...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro de nosotros fuimos recientemente a Eddi...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta y simple: limpieza\n- bad. Tengo...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,Al reservar un hotel con multipropiedad Mayan ...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"No pierdan su tiempo ni dinero, venimos porque...",1,Hotel
5,El peor huevos Benedict jamás,Hoy teníamos el desayuno por segunda vez en po...,1,Restaurant
6,Definitivamente no volvería a hospedarme en Sa...,"El hotel en si no es malo, pero mi experiencia...",1,Hotel
7,Terrible,No estoy seguro de por qué este restaurante ti...,1,Restaurant
8,"Bebidas ADULTERADAS, FATAL mi experiencia!!!",Llegué a este hotel “por desgracia” mi reserva...,1,Hotel
9,Hotel ha ido cuesta abajo,Hemos estado viniendo a Villa la Estancia dura...,1,Hotel


In [3]:
data['Opinion'][7693]

'Aunque me parece algo hipócrita el arte de Rivera (todo lo ve desde fuera desde una zona segura), debo reconocer que tiene algunas obras que me agradan. Este museo es interesante por el simple hecho de conocer el primer hogar que fuera de Rivera. Bastante fancy. Está chido si te gusta ver cosas antiguas.'

In [62]:
myTest = data.head(10000)
print(myTest)

                                      Title  \
0                              Pésimo lugar   
1                 No vayas a lugar de Eddie   
2              Mala relación calidad-precio   
3          Minusválido? ¡No te alojes aquí!   
4     Es una porqueria no pierdan su tiempo   
...                                     ...   
9995                               Vacacion   
9996                 Excelente y muy barato   
9997                      Mi lugar favorito   
9998                     Joysquad excelente   
9999               El hotel y sus empleados   

                                                Opinion  Polarity  Attraction  
0     Piensen dos veces antes de ir a este hotel, te...         1       Hotel  
1     Cuatro de nosotros fuimos recientemente a Eddi...         1  Restaurant  
2     seguiré corta y simple: limpieza\n- bad. Tengo...         1       Hotel  
3     Al reservar un hotel con multipropiedad Mayan ...         1       Hotel  
4     No pierdan su tiempo ni diner

# Using Sentiment lexicons

In [46]:
def load_sel():
	#~ global lexicon_sel
	lexicon_sel = {}
	input_file = open('SEL_full.txt', 'r', encoding="utf-8")
	for line in input_file:
		#Las líneas del lexicon tienen el siguiente formato:
		#abundancia	0	0	50	50	0.83	Alegría
		
		palabras = line.split("\t")
		palabras[6]= re.sub('\n', '', palabras[6])
		pair = (palabras[6], palabras[5])
		if lexicon_sel:
			if palabras[0] not in lexicon_sel:
				lista = [pair]
				lexicon_sel[palabras[0]] = lista
			else:
				lexicon_sel[palabras[0]].append (pair)
		else:
			lista = [pair]
			lexicon_sel[palabras[0]] = lista
	input_file.close()
	del lexicon_sel['Palabra']; #Esta llave se inserta porque es parte del encabezado del diccionario, por lo que se requiere eliminar
	#Estructura resultante
		#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	return lexicon_sel

In [47]:
if (os.path.exists('lexicon_sel.pkl')):
    lexicon_sel_file = open ('lexicon_sel.pkl','rb')
    lexicon_sel = pickle.load(lexicon_sel_file)
else:
    lexicon_sel = load_sel()
    lexicon_sel_file = open ('lexicon_sel.pkl','wb')
    pickle.dump(lexicon_sel, lexicon_sel_file)
    lexicon_sel_file.close()

In [48]:
print(lexicon_sel['gustar'])

[('Alegría', '0.596')]


In [73]:
def getSELFeatures(cadenas, lexicon_sel):
	#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	features = []
	for cadena in cadenas:
		valor_alegria = 0.0
		valor_enojo = 0.0
		valor_miedo = 0.0
		valor_repulsion = 0.0
		valor_sorpresa = 0.0
		valor_tristeza = 0.0
		if(type(cadena)!=type("h")):
			cadena = ""
		cadena_palabras = re.split('\s+', cadena)
		dic = {}
		for palabra in cadena_palabras:
			if palabra in lexicon_sel:
				print("p: "+palabra)

				caracteristicas = lexicon_sel[palabra]
				for emocion, valor in caracteristicas:
					if emocion == 'Alegría':
						valor_alegria = valor_alegria + float(valor)
					elif emocion == 'Tristeza':
						valor_tristeza = valor_tristeza + float(valor)
					elif emocion == 'Enojo':
						valor_enojo = valor_enojo + float(valor)
					elif emocion == 'Repulsión':
						valor_repulsion = valor_repulsion + float(valor)
					elif emocion == 'Miedo':
						valor_miedo = valor_miedo + float(valor)
					elif emocion == 'Sorpresa':
						valor_sorpresa = valor_sorpresa + float(valor)
		dic['__txt__'] = cadena_palabras
		dic['__alegria__'] = valor_alegria
		dic['__tristeza__'] = valor_tristeza
		dic['__enojo__'] = valor_enojo
		dic['__repulsion__'] = valor_repulsion
		dic['__miedo__'] = valor_miedo
		dic['__sorpresa__'] = valor_sorpresa
		
		#Esto es para los valores acumulados del mapeo a positivo (alegría + sorpresa) y negativo (enojo + miedo + repulsión + tristeza)
		dic['acumuladopositivo'] = dic['__alegria__'] + dic['__sorpresa__']
		dic['acumuladonegative'] = dic['__enojo__'] + dic['__miedo__'] + dic['__repulsion__'] + dic['__tristeza__']
		
		features.append (dic)
	
	
	return features

In [70]:
myTest

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"piensir hotel , molestar , hijo año traer infl...",1,Hotel
1,No vayas a lugar de Eddie,recientemente Eddie's Place . servicio terribl...,1,Restaurant
2,Mala relación calidad-precio,seguir corto simple : limpieza \n - bad . insi...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo deci...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"perder tiempo dinero , venir tener boda , comp...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio calidad ...",5,Hotel
9996,Excelente y muy barato,"carne , tortilla recién hecho , queso fundido ...",5,Restaurant
9997,Mi lugar favorito,", seguido , comer delicioso atender maravilla ...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


In [71]:
polaridades = getSELFeatures(myTest['Opinion'], lexicon_sel)

polaridades_cadenas = []

# Bucle para procesar cada cadena
for i in range(len(polaridades)):
    polaridad_pos = np.array([polaridades[i]['acumuladopositivo']])
    polaridad_neg = np.array([polaridades[i]['acumuladonegative']])
    polaridad_cadena = np.concatenate((polaridad_pos, polaridad_neg), axis=0)
    polaridades_cadenas.append(polaridad_cadena)
    
    # print(f"POLARIDAD CADENA {i + 1}:")
    # print(polaridad_cadena)

# Convertir la lista de polaridades de cadenas a un array
polaridades_cadenas = np.stack(polaridades_cadenas)

print('Polaridades:')
print("POLARIDADES CADENAS:")
print(polaridades_cadenas)
print(" len: ")
print(len(polaridades_cadenas))
# Concatenar las polaridades con el array X
# test_con_lexicon = hstack([myTest, polaridades_cadenas]).toarray()
polaridades_df = pd.DataFrame(polaridades_cadenas, columns=['polaridad_positiva', 'polaridad_negativa'])
test_con_lexicon = pd.concat([myTest, polaridades_df],axis=1)

print('Vectorizado + polaridad:')


Polaridades:
POLARIDADES CADENAS:
[[0.    2.521]
 [0.    2.02 ]
 [0.    0.   ]
 ...
 [1.528 0.   ]
 [2.986 0.   ]
 [3.49  0.099]]
 len: 
10000
Vectorizado + polaridad:


In [72]:
test_con_lexicon

,Title,Opinion,Polarity,Attraction,polaridad_positiva,polaridad_negativa
0,Pésimo lugar,"piensir hotel , molestar , hijo año traer infl...",1,Hotel,0.000,2.521
1,No vayas a lugar de Eddie,recientemente Eddie's Place . servicio terribl...,1,Restaurant,0.000,2.020
2,Mala relación calidad-precio,seguir corto simple : limpieza \n - bad . insi...,1,Hotel,0.000,0.000
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo deci...,1,Hotel,2.354,0.561
4,Es una porqueria no pierdan su tiempo,"perder tiempo dinero , venir tener boda , comp...",1,Hotel,0.795,2.751
...,...,...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio calidad ...",5,Hotel,1.525,0.000
9996,Excelente y muy barato,"carne , tortilla recién hecho , queso fundido ...",5,Restaurant,0.330,0.000
9997,Mi lugar favorito,", seguido , comer delicioso atender maravilla ...",5,Restaurant,1.528,0.000
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel,2.986,0.000


In [74]:
getSELFeatures([test_con_lexicon['Opinion'][0]],lexicon_sel)

p: molestar
p: llorar
p: molestar


[{'__txt__': ['piensir',
   'hotel',
   ',',
   'molestar',
   ',',
   'hijo',
   'año',
   'traer',
   'inflable',
   'tamaño',
   'pequeño',
   'decir',
   'ver',
   'llorar',
   'seguir',
   'molestar',
   ',',
   'traíir',
   'hielera',
   'bolso',
   'rueda',
   'estilo',
   'Tmb',
   'molestándono',
   'bebida',
   'carísimo',
   'agua',
   'dejar',
   'alberca',
   'huésped',
   'hotel',
   'mayan',
   'palace',
   'grand',
   'mayan',
   'poder',
   'jamás',
   'volver',
   'venir',
   'lugar',
   ',',
   'invitar',
   'cena',
   'show',
   'mil',
   'pesos',
   'persona',
   'cena',
   'dogs',
   ',',
   'habitación',
   'normal',
   ''],
  '__alegria__': 0.0,
  '__tristeza__': 1.129,
  '__enojo__': 1.392,
  '__repulsion__': 0.0,
  '__miedo__': 0.0,
  '__sorpresa__': 0.0,
  'acumuladopositivo': 0.0,
  'acumuladonegative': 2.521}]

In [75]:
test_con_lexicon.head(1)

,Title,Opinion,Polarity,Attraction,polaridad_positiva,polaridad_negativa
0,Pésimo lugar,"piensir hotel , molestar , hijo año traer infl...",1,Hotel,0.0,2.521


# Normalizacion

In [63]:
nlp = spacy.load("es_core_news_sm")

In [64]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

def lemmatize_token(token):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(token)

# nltk:
def tokenizarNLTK(doc):
    if type(doc) != type(""):
        print(doc)
        return ""

    # Tokenize the document
    tokens = word_tokenize(doc, language='spanish')

    # Remove stop words and lemmatize
    stop_words = set(stopwords.words('spanish'))
    filtered_tokens = [lemmatize_token(token) for token in tokens if token.lower() not in stop_words]

    # Join the filtered tokens into a string
    res = " ".join(filtered_tokens)

    return res

# spacy: 
def tokenizarSpacy(doc): 
    if(type(doc) != type("")):
         print(doc)
         return ""
    # nlp = spacy.load("es_core_news_sm") # español
    
    tkDoc = nlp(doc)
    res = ""
    for token in tkDoc:
        if(not(token.is_stop)): # quitando stop words
                res+=token.lemma_
                res+=" "

    return res

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sergi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [65]:
myTest['Opinion'] = myTest['Opinion'].apply(tokenizarSpacy)

nan


C:\Users\sergi\AppData\Local\Temp\ipykernel_2896\3556711929.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['Opinion'] = myTest['Opinion'].apply(tokenizarSpacy)


In [56]:
myTest

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"Piensen do veces ir hotel , molestan , hijo se...",1,Hotel
1,No vayas a lugar de Eddie,Cuatro recientemente Eddie 's Place . servicio...,1,Restaurant
2,Mala relación calidad-precio,seguiré corta simple : limpieza - bad . insist...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo dije...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"pierdan tiempo dinero , venimos boda aquí mism...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio primera ...",5,Hotel
9996,Excelente y muy barato,"Buena carne , tortilla recién hechas , queso f...",5,Restaurant
9997,Mi lugar favorito,"Siempre vamos , vamos seguido , comemos delici...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


In [9]:
myTest['Polarity'].value_counts()

Polarity
4    5878
3    2121
2     730
5     724
1     547
Name: count, dtype: int64

In [25]:
result = myTest[myTest['Opinion']=="error"]

In [26]:
result

,Title,Opinion,Polarity,Attraction


In [27]:
myTest['Title'] = myTest['Title'].apply(tokenizar)

2016-12-01 00:00:00
2015-05-01 00:00:00


C:\Users\sergi\AppData\Local\Temp\ipykernel_21960\745785095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myTest['Title'] = myTest['Title'].apply(tokenizar)


In [76]:
myTest

,Title,Opinion,Polarity,Attraction
0,Pésimo lugar,"piensir hotel , molestar , hijo año traer infl...",1,Hotel
1,No vayas a lugar de Eddie,recientemente Eddie's Place . servicio terribl...,1,Restaurant
2,Mala relación calidad-precio,seguir corto simple : limpieza \n - bad . insi...,1,Hotel
3,Minusválido? ¡No te alojes aquí!,reservar hotel multipropiedad Mayan grupo deci...,1,Hotel
4,Es una porqueria no pierdan su tiempo,"perder tiempo dinero , venir tener boda , comp...",1,Hotel
...,...,...,...,...
9995,Vacacion,"excelente lugar vacacionar , servicio calidad ...",5,Hotel
9996,Excelente y muy barato,"carne , tortilla recién hecho , queso fundido ...",5,Restaurant
9997,Mi lugar favorito,", seguido , comer delicioso atender maravilla ...",5,Restaurant
9998,Joysquad excelente,Lisbhet Torre Punta excelente servicio amable ...,5,Hotel


# Splitting

In [77]:
train, test = train_test_split(test_con_lexicon, test_size=0.2)

In [78]:
train

,Title,Opinion,Polarity,Attraction,polaridad_positiva,polaridad_negativa
5585,Ame Samba Vallarta,recordar amable segmentado edificio dar privac...,4,Hotel,4.010,0.000
5780,Hard Rock luna,marido celebrar luna miel Hard Rock Vallarta r...,4,Hotel,7.010,2.413
4506,Fin de semana de Memorial,alojar 4 pasar . conserje Laticia ! servicio g...,4,Hotel,2.060,1.025
8402,Las Momias de Guanajuato,"sala , poder apreciar número momia , pequeño e...",4,Attractive,0.629,0.000
1052,» Wannabe de comida mejicana »!!,decepcionante refugié altamente lugar . peor c...,2,Restaurant,1.791,2.978
...,...,...,...,...,...,...
6827,Festejo familiar,excelente hotel familiar . servicio personal e...,4,Hotel,4.311,0.000
6712,Buena comida,fiable fiable . comida servicio . pasta pizza ...,4,Restaurant,0.864,0.000
5991,Restaurantes asiáticos,precioso entorno estanque espejo estanque natu...,4,Restaurant,2.192,1.588
7537,"Muy buen hotel, para regresar a él.",excelente hotel ; instalación limpio ; mayoría...,4,Hotel,3.753,0.000


In [79]:
test

,Title,Opinion,Polarity,Attraction,polaridad_positiva,polaridad_negativa
8560,Hermosa vista,monumento llegar carro funicular . enorme esta...,4,Attractive,1.131,0.000
115,decepcionado,"noviembre , Lalaguna , práctico hotel agradabl...",1,Restaurant,1.097,0.000
2302,No tan luxxe,estar verano viaje Puerto Vallarta . estábamos...,3,Hotel,1.493,0.000
6030,Buena relación calidad-precio.,playa agradable . comida excelente . bebida . ...,4,Hotel,4.381,0.000
362,El Hotel En Mantenimiento y Musica hasta las 1...,"Acabo pasar semana suite hotel , visitado gene...",1,Hotel,3.977,0.994
...,...,...,...,...,...,...
5294,Gran lugar; ruido terrible,"hotel año seguido , ambos ocasión alogir apart...",4,Hotel,5.011,0.796
6758,Muy bonito complejo,marido alogir Villa Palmar 1 Flamingos complej...,4,Hotel,11.542,4.943
2912,Tradicional,"lugar bonito , cubrir expectativa generado . f...",3,Attractive,0.297,0.000
9769,En familia en la playa,"limpieza impecable , tamaño habitación cómodo ...",5,Hotel,1.292,0.099


# Text Representations

In [80]:
vectorizador_binario = CountVectorizer(binary=True, token_pattern= r'(?u)\w\w+|\w\w+\n|\.')
vectorizador_frecuencia = CountVectorizer(token_pattern= r'(?u)\w\w+|\w\w+\n|\.')
vectorizador_tfidf = TfidfVectorizer()

In [81]:
train_vectorizado = vectorizador_tfidf.fit_transform(train['Opinion'])

In [83]:
train_vectorizado

<8000x23452 sparse matrix of type '<class 'numpy.float64'>'
	with 352771 stored elements in Compressed Sparse Row format>

In [84]:
test_vectorizado = vectorizador_tfidf.transform(test['Opinion'])

In [85]:
print(vectorizador_tfidf.get_feature_names_out())
print(train_vectorizado.toarray())
print(len(train_vectorizado.toarray()[0]))
print(len(vectorizador_tfidf.get_feature_names_out()))

['00' '000' '00am' ... 'útil' 'útlimo' 'ĺlegar']
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
23452
23452


# Training ML models

## Without NLTK

In [90]:
x_train = train_vectorizado.toarray()
y_train = train['Polarity']
x_test = test_vectorizado
y_test = test['Polarity']


In [124]:
print(x_train)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [122]:
print(type(train[['polaridad_positiva','polaridad_negativa']].values))

<class 'numpy.ndarray'>


In [123]:
print(train[['polaridad_positiva','polaridad_negativa']].values)

[[4.01  0.   ]
 [7.01  2.413]
 [2.06  1.025]
 ...
 [2.192 1.588]
 [3.753 0.   ]
 [4.822 1.588]]


In [134]:

polaridad = train[['polaridad_positiva','polaridad_negativa']].values

print("Shape of x_train:", x_train.shape)
print("Shape of polaridad:", polaridad.shape)

from scipy.sparse import csr_matrix

x_train_sparse = csr_matrix(x_train)
polaridad_sparse = csr_matrix(polaridad)

x_train_lexicon = hstack((x_train_sparse, polaridad_sparse)).toarray()
print(x_train_lexicon)

Shape of x_train: (8000, 23452)
Shape of polaridad: (8000, 2)
[[0.    0.    0.    ... 0.    4.01  0.   ]
 [0.    0.    0.    ... 0.    7.01  2.413]
 [0.    0.    0.    ... 0.    2.06  1.025]
 ...
 [0.    0.    0.    ... 0.    2.192 1.588]
 [0.    0.    0.    ... 0.    3.753 0.   ]
 [0.    0.    0.    ... 0.    4.822 1.588]]


In [135]:
polaridad_test = test[['polaridad_positiva','polaridad_negativa']].values

print("Shape of x_test:", x_test.shape)
print("Shape of polaridad_test:", polaridad_test.shape)

x_test_sparse = csr_matrix(x_test)
polaridad_test_sparse = csr_matrix(polaridad_test)

x_test_lexicon = hstack((x_test_sparse, polaridad_test_sparse)).toarray()
print(x_test_lexicon)

Shape of x_test: (2000, 23452)
Shape of polaridad_test: (2000, 2)
[[0.    0.    0.    ... 0.    1.131 0.   ]
 [0.    0.    0.    ... 0.    1.097 0.   ]
 [0.    0.    0.    ... 0.    1.493 0.   ]
 ...
 [0.    0.    0.    ... 0.    0.297 0.   ]
 [0.    0.    0.    ... 0.    1.292 0.099]
 [0.    0.    0.    ... 0.    0.594 0.033]]


In [137]:
clf_polarity = LogisticRegression(max_iter=10000)
clf_polarity.fit(x_train_lexicon,y_train)
y_pred = clf_polarity.predict(x_test_lexicon)

In [17]:
clf_bayes = MultinomialNB()
clf_bayes.fit(x_train, y_train)
y_pred = clf_bayes.predict(x_test)

### Lematizacion, stop words. Binarizado. Logistic Regression:

In [44]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Frecuencia. Logistic Regression:

In [45]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Tf-idf. Logistic Regression:

In [46]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Lematizacion, stop words. Tf-idf. Multinomial Naive Bayes

In [47]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00       110
           2       0.00      0.00      0.00       136
           3       0.00      0.00      0.00       446
           4       0.58      1.00      0.73      1154
           5       0.00      0.00      0.00       154

    accuracy                           0.58      2000
   macro avg       0.12      0.20      0.15      2000
weighted avg       0.33      0.58      0.42      2000



c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\sergi\miniconda3\envs\cursoNLP\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## With NLTK

### Lematizacion, stop words. Binarizado. Logistic Regression:

In [24]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.68      0.21      0.32       109
           2       0.44      0.05      0.09       143
           3       0.43      0.27      0.33       409
           4       0.69      0.95      0.80      1201
           5       0.74      0.10      0.18       138

    accuracy                           0.65      2000
   macro avg       0.59      0.32      0.34      2000
weighted avg       0.62      0.65      0.58      2000



### Lematizacion, stop words. Frecuencia. Logistic Regression:

In [138]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.48      0.19      0.28       113
           2       0.33      0.06      0.11       144
           3       0.40      0.27      0.32       400
           4       0.68      0.94      0.79      1187
           5       0.77      0.13      0.22       156

    accuracy                           0.63      2000
   macro avg       0.53      0.32      0.34      2000
weighted avg       0.59      0.63      0.57      2000



In [139]:
cv_results = cross_validate(clf_polarity, x_train,y_train, cv=5, scoring='f1_macro', verbose = 3, n_jobs = -1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [ ]:
cv_results['test_score'].mean()

0.27621456039405234